<font size=6><b>펀드 운용모델
* 참고: 실습06_펀드_운용모델.xlsx

In [223]:
from pykrx import stock
from pykrx import bond
import FinanceDataReader as fdr

from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time


import warnings
warnings.filterwarnings(action='ignore')

sns.set()
#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

# Data Load

## 시총 Top 5 업종명, 업종코드
* LG에너지솔루션, 삼성전자우 제외

In [224]:
start_data='20200101'
end_data='20221229'

df = stock.get_market_cap(end_data) 
top5_ticker = df.sort_values(by='시가총액', ascending=False).index[:7]
top5_name = []
for ticker in top5_ticker:
    names = stock.get_market_ticker_name(ticker)
    top5_name.append(names)
    print(ticker, names)
    time.sleep(1)  
    
top5_ticker = list(top5_ticker)

top5_ticker.remove('373220')
top5_ticker.remove('005935')

top5_name.remove('LG에너지솔루션')
top5_name.remove('삼성전자우')

print(len(top5_ticker), top5_ticker)
print(len(top5_name), top5_name)

005930 삼성전자
373220 LG에너지솔루션
207940 삼성바이오로직스
000660 SK하이닉스
051910 LG화학
005935 삼성전자우
006400 삼성SDI
5 ['005930', '207940', '000660', '051910', '006400']
5 ['삼성전자', '삼성바이오로직스', 'SK하이닉스', 'LG화학', '삼성SDI']


## 종목 기본정보
* 상장주식수, 유동비율
* fnguide 크롤링

In [225]:
from bs4 import BeautifulSoup
import requests
import re

In [226]:
발생주식수_list = []
유동비율_list = []
유동주식수_list = []

for ticker in top5_ticker:
    res = requests.get(f"https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{ticker}&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN=")
    soup = BeautifulSoup(res.text, 'html.parser')
    
    trdata = soup.select("#svdMainGrid1> table > tbody > tr")
    td = trdata[5].select("td")
    
    발행주식수 = td[0].text
    발행주식수 = 발행주식수.split("/")[0].strip()
    발행주식수 = int(''.join(re.findall('[\d.]+', 발행주식수)))
    
    유동비율 = td[1].text
    유동비율 = 유동비율.split('/')[1].strip()
    유동비율= float(유동비율)
    
    유동주식수 = td[1].text
    유동주식수 = 유동주식수.split('/')[0].strip()
    유동주식수 = int(''.join(re.findall('[\d.]+', 유동주식수)))
    
    발생주식수_list.append(발행주식수)
    유동비율_list.append(유동비율)
    유동주식수_list.append(유동주식수)
    
    print(발행주식수, 유동비율, 유동주식수)
    
    time.sleep(1)

5969782550 75.81 4525870769
71174000 24.76 17623664
728002365 73.87 537759511
70592343 66.63 47036786
68764530 73.16 50306427


In [227]:
발생주식수_list

[5969782550, 71174000, 728002365, 70592343, 68764530]

In [228]:
유동비율_list

[75.81, 24.76, 73.87, 66.63, 73.16]

In [229]:
유동주식수_list  

[4525870769, 17623664, 537759511, 47036786, 50306427]

## 일자별 주가

In [230]:
주가_list = []
for ticker in top5_ticker:
    names = stock.get_market_ticker_name(ticker)
    closev = stock.get_market_ohlcv(start_data, end_data, ticker)[['종가']]
    closev.columns = [names]
    주가_list.append(closev)
    time.sleep(1)  
    
주가_df = pd.concat(주가_list, axis=1)  
주가_df.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI
날짜,,,,,
2020-01-02,55200,423485,94700,314000,232000
2020-01-03,55500,417061,94500,311000,229000
2020-01-06,55500,407178,94300,307000,230000
2020-01-07,55800,414096,94000,311000,231500
2020-01-08,56800,403719,97400,302000,229000


In [245]:
주가_df.to_csv('../datasets/일자별주가.csv')

# 일자별 시가 총액

In [231]:
# 유동주식수 = np.array(발생주식수_list) * np.array(유동비율_list)*0.01
# 시가총액_df = 주가_df*유동주식수
시가총액_df = 주가_df*np.array(유동주식수_list)
시가총액_df.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI
날짜,,,,,
2020-01-02,249828066448800,7463357349040,50925825691700,14769550804000,11671091064000
2020-01-03,251185827679500,7350142931504,50818273789500,14628440446000,11520171783000
2020-01-06,251185827679500,7175968260192,50710721887300,14440293302000,11570478210000
2020-01-07,252543588910200,7297888767744,50549394034000,14628440446000,11645937850500
2020-01-08,257069459679200,7115008006416,52377776371400,14205109372000,11520171783000


In [232]:
# 시가총액_df['합계'] = 시가총액_df.iloc[:,0]+시가총액_df.iloc[:,1]+시가총액_df.iloc[:,2]+시가총액_df.iloc[:,3]+시가총액_df.iloc[:,4]
시가총액_df['합계'] = 시가총액_df.sum(axis=1)
시가총액_df.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI,합계
날짜,,,,,,
2020-01-02,249828066448800,7463357349040,50925825691700,14769550804000,11671091064000,334657891357540
2020-01-03,251185827679500,7350142931504,50818273789500,14628440446000,11520171783000,335502856629504
2020-01-06,251185827679500,7175968260192,50710721887300,14440293302000,11570478210000,335083289338992
2020-01-07,252543588910200,7297888767744,50549394034000,14628440446000,11645937850500,336665250008444
2020-01-08,257069459679200,7115008006416,52377776371400,14205109372000,11520171783000,342287525212016


In [233]:
시가총액_df['지수'] = 시가총액_df['합계']/시가총액_df.iloc[0,5]
시가총액_df.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI,합계,지수
날짜,,,,,,,
2020-01-02,249828066448800,7463357349040,50925825691700,14769550804000,11671091064000,334657891357540,1.000000
2020-01-03,251185827679500,7350142931504,50818273789500,14628440446000,11520171783000,335502856629504,1.002525
2020-01-06,251185827679500,7175968260192,50710721887300,14440293302000,11570478210000,335083289338992,1.001271
2020-01-07,252543588910200,7297888767744,50549394034000,14628440446000,11645937850500,336665250008444,1.005998
2020-01-08,257069459679200,7115008006416,52377776371400,14205109372000,11520171783000,342287525212016,1.022798


# 펀드

## 편입비중

In [234]:
편입비중_df = 시가총액_df[top5_name]
편입비중_df = 편입비중_df.div(시가총액_df['합계'], axis=0)
편입비중_df.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI
날짜,,,,,
2020-01-02,0.746518,0.022301,0.152173,0.044133,0.034875
2020-01-03,0.748685,0.021908,0.151469,0.043602,0.034337
2020-01-06,0.749622,0.021415,0.151338,0.043095,0.034530
2020-01-07,0.750133,0.021677,0.150147,0.043451,0.034592
2020-01-08,0.751034,0.020787,0.153023,0.041501,0.033656


## 설정좌수, 환매좌수

In [235]:
좌수 = pd.read_csv('../datasets/실습06_설정환매_랜덤.csv', parse_dates=['날짜'])
좌수 = 좌수.set_index('날짜')
좌수.head()

,설정,환매
날짜,,
2020-01-02,1000000,0
2020-01-03,50000,0
2020-01-06,150000,100000
2020-01-07,100000,100000
2020-01-08,150000,100000


In [236]:
좌수['총좌수'] = (좌수['설정']-좌수['환매']).cumsum()
좌수.head()

,설정,환매,총좌수
날짜,,,
2020-01-02,1000000,0,1000000
2020-01-03,50000,0,1050000
2020-01-06,150000,100000,1100000
2020-01-07,100000,100000,1100000
2020-01-08,150000,100000,1150000


## iNAV

In [246]:
기준가 = 10000
iNAV = (시가총액_df['지수']/시가총액_df['지수'][0])*기준가
# iNAV = iNAV.astype(int)
iNAV

날짜
2020-01-02    10000.000000
2020-01-03    10025.248628
2020-01-06    10012.711428
2020-01-07    10059.982409
2020-01-08    10227.983085
                  ...     
2022-12-23    11324.355459
2022-12-26    11307.129031
2022-12-27    11351.391665
2022-12-28    11065.406784
2022-12-29    10847.938277
Name: 지수, Length: 742, dtype: float64

## AUM

In [247]:
AUM = iNAV*좌수['총좌수']
AUM

날짜
2020-01-02    1.000000e+10
2020-01-03    1.052651e+10
2020-01-06    1.101398e+10
2020-01-07    1.106598e+10
2020-01-08    1.176218e+10
                  ...     
2022-12-23    4.240971e+11
2022-12-26    4.240173e+11
2022-12-27    4.268123e+11
2022-12-28    4.166126e+11
2022-12-29    4.089673e+11
Length: 742, dtype: float64

## 일자별 매매내역

In [248]:
주식잔고_df = 편입비중_df[top5_name]

주식잔고_df = 주식잔고_df.mul(AUM, axis=0)
주식잔고_df = 주식잔고_df.div(주가_df)

주식잔고_df = 주식잔고_df.astype(int)

주식잔고_df.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI
날짜,,,,,
2020-01-02,135238,526,16068,1405,1503
2020-01-03,142000,552,16872,1475,1578
2020-01-06,148762,579,17675,1546,1653
2020-01-07,148762,579,17675,1546,1653
2020-01-08,155524,605,18479,1616,1728


In [249]:
매매내역_df = 주식잔고_df.diff()
매매내역_df.iloc[0,:] = 주식잔고_df.iloc[0,:]
매매내역_df.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI
날짜,,,,,
2020-01-02,135238.0,526.0,16068.0,1405.0,1503.0
2020-01-03,6762.0,26.0,804.0,70.0,75.0
2020-01-06,6762.0,27.0,803.0,71.0,75.0
2020-01-07,0.0,0.0,0.0,0.0,0.0
2020-01-08,6762.0,26.0,804.0,70.0,75.0


## Total Fund Result

In [250]:
fdf = pd.concat([편입비중_df,시가총액_df['지수'],  좌수, iNAV, AUM, 매매내역_df, 주식잔고_df], axis=1)
fdf.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI,지수,설정,환매,총좌수,지수,0,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI
날짜,,,,,,,,,,,,,,,,,,,,,
2020-01-02,0.746518,0.022301,0.152173,0.044133,0.034875,1.000000,1000000,0,1000000,10000.000000,1.000000e+10,135238.0,526.0,16068.0,1405.0,1503.0,135238,526,16068,1405,1503
2020-01-03,0.748685,0.021908,0.151469,0.043602,0.034337,1.002525,50000,0,1050000,10025.248628,1.052651e+10,6762.0,26.0,804.0,70.0,75.0,142000,552,16872,1475,1578
2020-01-06,0.749622,0.021415,0.151338,0.043095,0.034530,1.001271,150000,100000,1100000,10012.711428,1.101398e+10,6762.0,27.0,803.0,71.0,75.0,148762,579,17675,1546,1653
2020-01-07,0.750133,0.021677,0.150147,0.043451,0.034592,1.005998,100000,100000,1100000,10059.982409,1.106598e+10,0.0,0.0,0.0,0.0,0.0,148762,579,17675,1546,1653
2020-01-08,0.751034,0.020787,0.153023,0.041501,0.033656,1.022798,150000,100000,1150000,10227.983085,1.176218e+10,6762.0,26.0,804.0,70.0,75.0,155524,605,18479,1616,1728


In [251]:
fdf.columns = ['편입비중_삼성전자', '편입비중_삼성바이오로직스', '편입비중_SK하이닉스', '편입비중_LG화학', '편입비중_삼성SDI',
              '지수', '설정좌수', '환매좌수', '총좌수', 'iNAV', 'AUM',
              '매매내역_삼성전자', '매매내역_삼성바이오로직스', '매매내역_SK하이닉스', '매매내역_LG화학', '매매내역_삼성SDI',
              '주식잔고_삼성전자', '주식잔고_삼성바이오로직스', '주식잔고_SK하이닉스', '주식잔고_LG화학', '주식잔고_삼성SDI']
fdf.head()

,편입비중_삼성전자,편입비중_삼성바이오로직스,편입비중_SK하이닉스,편입비중_LG화학,편입비중_삼성SDI,지수,설정좌수,환매좌수,총좌수,iNAV,AUM,매매내역_삼성전자,매매내역_삼성바이오로직스,매매내역_SK하이닉스,매매내역_LG화학,매매내역_삼성SDI,주식잔고_삼성전자,주식잔고_삼성바이오로직스,주식잔고_SK하이닉스,주식잔고_LG화학,주식잔고_삼성SDI
날짜,,,,,,,,,,,,,,,,,,,,,
2020-01-02,0.746518,0.022301,0.152173,0.044133,0.034875,1.000000,1000000,0,1000000,10000.000000,1.000000e+10,135238.0,526.0,16068.0,1405.0,1503.0,135238,526,16068,1405,1503
2020-01-03,0.748685,0.021908,0.151469,0.043602,0.034337,1.002525,50000,0,1050000,10025.248628,1.052651e+10,6762.0,26.0,804.0,70.0,75.0,142000,552,16872,1475,1578
2020-01-06,0.749622,0.021415,0.151338,0.043095,0.034530,1.001271,150000,100000,1100000,10012.711428,1.101398e+10,6762.0,27.0,803.0,71.0,75.0,148762,579,17675,1546,1653
2020-01-07,0.750133,0.021677,0.150147,0.043451,0.034592,1.005998,100000,100000,1100000,10059.982409,1.106598e+10,0.0,0.0,0.0,0.0,0.0,148762,579,17675,1546,1653
2020-01-08,0.751034,0.020787,0.153023,0.041501,0.033656,1.022798,150000,100000,1150000,10227.983085,1.176218e+10,6762.0,26.0,804.0,70.0,75.0,155524,605,18479,1616,1728


In [252]:
fdf.tail()

,편입비중_삼성전자,편입비중_삼성바이오로직스,편입비중_SK하이닉스,편입비중_LG화학,편입비중_삼성SDI,지수,설정좌수,환매좌수,총좌수,iNAV,AUM,매매내역_삼성전자,매매내역_삼성바이오로직스,매매내역_SK하이닉스,매매내역_LG화학,매매내역_삼성SDI,주식잔고_삼성전자,주식잔고_삼성바이오로직스,주식잔고_SK하이닉스,주식잔고_LG화학,주식잔고_삼성SDI
날짜,,,,,,,,,,,,,,,,,,,,,
2022-12-23,0.693847,0.037481,0.110396,0.075710,0.082566,1.132436,50000,0,37450000,11324.355459,4.240971e+11,6762.0,26.0,803.0,70.0,75.0,5064690,19721,601781,52636,56295
2022-12-26,0.692512,0.038284,0.109427,0.076820,0.082957,1.130713,150000,100000,37500000,11307.129031,4.240173e+11,6762.0,27.0,803.0,70.0,75.0,5071452,19748,602584,52706,56370
2022-12-27,0.692194,0.038413,0.109001,0.077758,0.082634,1.135139,200000,100000,37600000,11351.391665,4.268123e+11,13524.0,52.0,1607.0,141.0,151.0,5084976,19800,604191,52847,56521
2022-12-28,0.691752,0.039501,0.110365,0.076466,0.081917,1.106541,100000,50000,37650000,11065.406784,4.166126e+11,6762.0,27.0,804.0,70.0,75.0,5091738,19827,604995,52917,56596
2022-12-29,0.689412,0.039856,0.111097,0.077739,0.081896,1.084794,100000,50000,37700000,10847.938277,4.089673e+11,6762.0,26.0,803.0,71.0,75.0,5098500,19853,605798,52988,56671
